In [1]:
!pip install langchain faiss-cpu openai unstructured pdfminer.six gradio tiktoken huggingface-hub langchain-community

In [7]:
from google.colab import files
uploaded = files.upload()  # Upload your medical/healthcare PDF


Saving PE_1_3_26.pdf to PE_1_3_26.pdf


In [8]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load PDF
loader = PyPDFLoader(next(iter(uploaded)))
documents = loader.load()

# Chunking
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = text_splitter.split_documents(documents)

In [9]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

embedding_model = HuggingFaceEmbeddings()
db = FAISS.from_documents(chunks, embedding_model)


<ipython-input-9-2396061899>:4: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding_model = HuggingFaceEmbeddings()


In [5]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(chunks, embedding_model)

In [10]:
# 🔧 Install all required libraries
!pip install langchain faiss-cpu transformers accelerate unstructured PyPDF2 gradio

# 🧠 Imports
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import pipeline
from google.colab import files
import gradio as gr

# 📄 Upload PDF
uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]

# 🧱 Step 1: Load & Chunk PDF
loader = PyPDFLoader(pdf_path)
docs = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = splitter.split_documents(docs)

# 🔍 Step 2: Embeddings + Vector DB
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(chunks, embedding_model)
retriever = vectorstore.as_retriever()

# 🤖 Step 3: Load Local LLM
llm_pipeline = pipeline("text2text-generation", model="google/flan-t5-base")

# 🔁 Step 4: Custom QA Function (No LangChain QA chain needed)
def ask_pdf(query):
    top_docs = retriever.get_relevant_documents(query)
    context = " ".join(doc.page_content for doc in top_docs[:3])
    prompt = f"Context: {context}\n\nQuestion: {query}\nAnswer:"
    result = llm_pipeline(prompt, max_length=256, do_sample=False)[0]["generated_text"]
    return result

# 💬 Step 5: Gradio Interface
gr.Interface(fn=ask_pdf,
             inputs=gr.Textbox(label="Ask a medical question about your PDF:"),
             outputs="text",
             title="🩺 Chat with Your Healthcare PDF (Local AI)",
             description="Uses FLAN-T5 and FAISS, no API key needed."
).launch()


Saving PE_1_3_26.pdf to PE_1_3_26 (1).pdf


Device set to use cpu


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://48f015c2f305a1d2c2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
